# Swell quantification

This notebook processes `output.csv` generated by `organoid_recognition.ipnyb`. This notebook contains several steps:

1. Mounting the Google Drive to access data
2. Installing and loading dependencies 
3. Defining functions used for post-processing
4. Combining recognized organoid data for all images
5. Setting parameters and testing post-processing, this cell performs:
  - Particle tracking
  - Linear Swell regression
  - Outlier removal based on standard error of slope parameter
6. Save output graph and output summary (with swell of each organoid
7. Aggregate data to obtain well statistics


Google Drive instructions
- Ensure that the shared `OrgaSwell` directory is placed in the Google Drive root
- Ensure that images to be quantified are placed within a subdirectory in `OrgaSwell/images`

## 1. Mount the Google Drive to access data

Follow the link and instructions.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Installing and loading dependencies 

In [2]:
%cd drive/My\ Drive/OrgaSwell

!pip install -q trackpy

import altair as alt
import trackpy as tp
import numpy as np
import pandas as pd
from pathlib import Path
import ipywidgets as widgets
import statsmodels.api as sm
from sklearn.preprocessing import RobustScaler

alt.data_transformers.disable_max_rows()

/content/drive/.shortcut-targets-by-id/1BUE89W79bFo9AAy4Ids63OyUwiYMHDUF/OrgaSwell


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


DataTransformerRegistry.enable('default')

## 3. Defining functions used for postprocessing

In [3]:
def graphical_well_summary(df):
  # Much of this code is duplicated in processing_impact_graph, can be rewritten in a single function

  well_nr = df['well'].values[0]

  output_filename = Path(df['parent_dir'].values[0]) / f'well_{well_nr}_graphical_summary'
  chart = alt.Chart(df).encode(color=alt.Color('particle:N', legend=None), tooltip=['particle:N'])
  tracking = chart.mark_point().encode(x=alt.X('x:Q', axis=alt.Axis(labels=False, ticks=False), title=None), 
                                       y=alt.Y('y:Q', axis=alt.Axis(labels=False, ticks=False), title=None))
  

  swell = chart.mark_line().encode(x='t:O', y=alt.Y('surface:Q', title='Surface (pixels^2)'))
  swell_fitted = chart.mark_point().encode(x='t:O', y=alt.Y('y_hat', title=None))


  df_to_plot_param = df[['well','particle','param_t']].drop_duplicates()
  df_to_plot_param['rank'] = df_to_plot_param.groupby('well')['param_t'].rank()

  bars = alt.Chart(df_to_plot_param).mark_bar()\
            .encode(x=alt.X('rank:O', title='particle counter'),
                    y=alt.Y('param_t:Q',title='Swell Rate (pixel^2/frame)'),
                    color=alt.Color('particle:N', legend=None),
                    tooltip=['particle:N'])

  (tracking | (swell + swell_fitted) | bars).save(str(output_filename) + '.html', format='html')

def linear_reg(df):

  y = df['surface']
  x = df['t']

  x = sm.add_constant(x)
  model = sm.OLS(y,x).fit()

  y_hat = model.predict(x)
  df['y_hat'] = y_hat

  params = model.params
  df['param_const'] = params['const']
  df['param_t'] = params['t']

  std_err = model.HC1_se
  df['param_const_se'] = std_err['const']
  df['param_t_se'] = std_err['t']
  return df

def processing_impact_graph(df, seed):

  # Select 3 wells to show impact of processing
  np.random.seed(seed)
  wells = np.random.choice(df['well'].unique(), 3, replace=False)
  df_to_plot = df[df['well'].isin(wells)]

  df_to_plot_param = df_to_plot[['well','particle','param_t']].drop_duplicates()
  df_to_plot_param['rank'] = df_to_plot_param.groupby('well')['param_t'].rank()

  chart = alt.Chart(df_to_plot, width=300, height=200)
  time_chart = chart.encode(x='t', color=alt.Color('particle:N',legend=None), 
                                  tooltip=['particle:N','param_t_se:Q'])
  point = time_chart.mark_line().encode(y='surface:Q')
  line = time_chart.mark_point().encode(y='y_hat')

  bars = alt.Chart(df_to_plot_param, width=300, height=200).mark_bar()\
            .encode(x=alt.X('rank:O', title='particle counter'),
                    y=alt.Y('param_t:Q',title='Swell Rate (pixel^2 / Frame)'),
                    color=alt.Color('particle:N', legend=None),
                    tooltip=['particle:N'])

  ((point + line).facet(row = 'well').resolve_scale(y='independent') | 
   bars.facet(row='well').resolve_scale(x='independent')).display()

def summary_graph(df_summ, output_dir):
  df_summ['ruler'] = 0

  chart = alt.Chart(df_summ).encode(x=alt.X('well:N'))

  error_bars = chart.mark_errorbar(extent='ci').encode(y=alt.Y('param_t:Q', title=''))

  points = chart.mark_point(filled=True, color='black')\
                .encode(y=alt.Y('param_t:Q', aggregate='mean', title='Swell rate (Pixel^2 / Frame)'),
                        tooltip=[alt.Tooltip('count(param_t)', title='n particles'), 'well'])
                
  annotation = chart.mark_text(align='left',
                             baseline='middle',
                             fontSize = 10,
                             dx = 5
                            ).encode(
                                y=alt.Y('param_t:Q', aggregate='mean', title=''),
                                text='count(param_t)')

  ruler = chart.mark_rule(color='r').encode(y='ruler:Q')

  graph = (error_bars + points + ruler + annotation)
  graph.display()
  graph.save(str(output_dir / 'summary.html'), format='html')

def scaling_graph(df_summ_well, output_dir):

  df_plot = df_summ_well.melt(id_vars='well', value_vars=['param_t','param_t_scaled'], value_name='param_t', var_name='scaling')

  graph = alt.Chart(df_plot).mark_bar(opacity=0.5).encode(x=alt.X('param_t', bin=alt.Bin(maxbins=100)),
                                      y=alt.Y('count(param_t)', stack=None),
                                      color='scaling')

  graph.display()
  graph.save(str(output_dir / 'swell_rate_scaling.html'), format='html')

# 4. Combining data for all images

Load csv's for each images and create output.csv for further processing. First we have to select the folder we want to analyze.

In [4]:
slider_path = widgets.Dropdown(options=[x.name for x in Path('./images').iterdir() if x.is_dir()],
                               description='Directory:',disabled=False)

display(slider_path)

Dropdown(description='Directory:', options=('113-plate2-FDA1-n2', '95W-95-plate2_Bright'), value='113-plate2-F…

In [5]:
folder_path = Path('./images') / slider_path.value

In [6]:
## Only run this cell if output.csv has not been created, creating it can take a long time for an experiment with many images

print('Saving combined output csv with all data')
# Combining all image .csv in an output .csv
target_files = [i for i in Path(folder_path).glob('**/*.csv') if 'output' not in str(i)]
df = pd.concat((pd.read_csv(file).assign(filename=file.stem, parent_dir=file.parent) for file in target_files), sort=False)
df['well'] = df['filename'].str.extract(r's(\d*)t').astype('int')
df['t'] = df['filename'].str.extract(r't(\d*)c').astype('int')
df['surface'] = 0.25 * df['Diameter 1 (Pixels)'] * df['Diameter 2 (Pixels)'] * np.pi

# Calculate centers
df['x'] = (df['x2'] + df['x1']) / 2
df['y'] = (df['y2'] + df['y1']) / 2
  
df.to_csv(folder_path / 'output.csv', sep=';', index=False)
print('Output is saved')

Saving combined output csv with all data


ValueError: ignored

## 5. Setting parameters and testing post-processing

Set parameters for particle tracking and linear regression outlier detection. Assess the set parameters based on the graph generated by the next cell.

In [ ]:
slider_memory  = widgets.IntSlider(value=2, min=0, max=13, description='Memory', 
                                        description_tooltip='Max number of consecutive frames a organoid can be undetected before reappering')

slider_min_t_steps  = widgets.IntSlider(value=10, min=0, max=13, description='Min t steps', 
                                        description_tooltip='Min number of time steps an organoid has to be observed')

slider_max_se  = widgets.IntSlider(value=11, min=5, max=25, description='Max Std Err', 
                                        description_tooltip='Max Std Error of fitted area incease parameter')

slider_seed = widgets.IntSlider(value=1, min=1, max=40, step=1, description='Seed',
                                    description_tooltip='Change value if parameter optimization should be performed on another image')

display(slider_memory, slider_min_t_steps, slider_max_se, slider_seed)

IntSlider(value=2, description='Memory', description_tooltip='Max number of consecutive frames a organoid can …

IntSlider(value=10, description='Min t steps', description_tooltip='Min number of time steps an organoid has t…

IntSlider(value=11, description='Max Std Err', description_tooltip='Max Std Error of fitted area incease param…

IntSlider(value=1, description='Seed', description_tooltip='Change value if parameter optimization should be p…

In [ ]:
# Parameters
memory = slider_memory.value
required_t_steps = slider_min_t_steps.value
max_se = slider_max_se.value
seed = slider_seed.value

# Load data
df = pd.read_csv(folder_path / 'output.csv', sep=';')

# Calculate centers
df['x'] = (df['x2'] + df['x1']) / 2
df['y'] = (df['y2'] + df['y1']) / 2

# Apply particle tracking
df = df.groupby('well').apply(tp.link, search_range=20, memory=memory, t_column='t').reset_index(drop=True)

# Filter for particles that have at least 10 timesteps
df = df[df.groupby(['well','particle'])['t'].transform('count') >= required_t_steps]

if len(df) == 0:
  raise Exception('Warning: no particles are left with these settings')

df = df.groupby(['well','particle']).apply(linear_reg)

# Remove particles with high t_se
df = df[df['param_t_se'] <= max_se]

processing_impact_graph(df, seed)

Frame 13: 47 trajectories present.


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


alt.HConcatChart(...)

## 6. Save output graph and output summary (with swell of each organoid)

In [ ]:
# Save summary graphs
df.groupby('well').apply(graphical_well_summary)

# Save output
df_summ = df[['well','particle','param_t']].drop_duplicates().copy()
df_summ.to_csv(folder_path / 'output_summary.csv', index=False, sep=';')

# Save summary graph
summary_graph(df_summ, folder_path)

alt.LayerChart(...)

# 7. Aggregate data to obtain well statistics

Next we create a summary with swell statistics for each well. To correct for scale deviations of the swell distribution that can occur due to unforseen different experimental conditions, the scaled swell rate is calculated using:  

$x_{scaled} = \dfrac{x-median(X)}{IQR}$


In [ ]:
df_summ_well = df_summ.groupby('well')['param_t'].mean().reset_index()

# Scale the swell rate
robustscaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0))
df_summ_well['param_t_scaled'] = robustscaler.fit_transform(df_summ_well['param_t'].values.reshape(-1, 1))

# Save the results
df_summ_well.to_csv(folder_path / 'well_output_summary.csv', index=False, sep=';')

# Visualize swell rate scaling
scaling_graph(df_summ_well, folder_path)

alt.Chart(...)